# Исследование надежности заемщиков


Проект выполнен для выявления влияния семейного положения и количества детей клиента на факт погашения кредита в срок. 

Цель: построение модели кредитного скоринга - специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Входные данные от банка — статистика о платёжеспособности клиентов.

План работы:
1. Рассмотреть данные
2. Провести их преодобработку
3. Категоризировать данные
4. Исследовать зависимость нескольких факторов и факта погашения кредита в срок
5. Сделать вывод

## Этап 1. Обзор данных

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Этап 2. Предобработка данных

### Удаление пропусков

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

`total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [ ]:
data['days_employed'] = data['days_employed'].abs()

Медианное значение трудового стажа для каждого типа занятости `days_employed` в днях.

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому осталвяем их как есть. Этот столбец не понадобится для исследования.

Перечень уникальных значений столбца `children`.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения.

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполнение пропусков в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Перечень уникальных целей взятия кредита из столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

`categorize_purpose` - функция, которая на основании данных из столбца `purpose` формирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Этап 3. Исследование данных и ответы на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
# Напишем функцию для перевода значений в проценты, функция будет
# актуальна для решения заданых вопросов

def per(i):
    return "{:.2%}".format(i)


In [ ]:
# Создадим сводную таблицу, в которую поместим данные о количестве детей (children) и
# данных о возврате кредитов в срок (debt). 

# В столбце debt методом sum посчитаем количество тех, кто не возвращает кредит в срок 
# затем методом count посчитаем общее количество людей с определенным количеством детей

# И воспользуемся методом mean, так как общее количество должников для каждого количества 
# детей разное

data_children = data.pivot_table(index='children', values='debt', aggfunc=('sum', 'count', 'mean'))
data_children

,count,mean,sum
children,,,
0,14091.0,0.075438,1063.0
1,4808.0,0.092346,444.0
2,2052.0,0.094542,194.0
3,330.0,0.081818,27.0
4,41.0,0.097561,4.0
5,9.0,0.000000,0.0


In [ ]:
# Переведем последний столбец в проценты с помощью написанной функции

data_children['mean'] = data_children['mean'].apply(per)

# Переименуем столбцы для лучшего восприятия и отсортируем столбцу со средним значением

data_children = data_children.rename(columns = {'sum': 'Количество должников', 'count': 'Общее количество', 'mean': 'Среднее по задолжностям'}) 
data_children.sort_values(by='Среднее по задолжностям', ascending=False)

,Общее количество,Среднее по задолжностям,Количество должников
children,,,
4,41.0,9.76%,4.0
2,2052.0,9.45%,194.0
1,4808.0,9.23%,444.0
3,330.0,8.18%,27.0
0,14091.0,7.54%,1063.0
5,9.0,0.00%,0.0


**Вывод:** 

Для семей с 4,5 детьми приведено критически мало данных, это выбросы, которые можно не учитывать при оценке зависимости.  Для семей с 3 детьми также мало данных, для учета этой категории нужно их добавить. 

В таком случае при оценке семей с 0-2 детьми можно сказать, что при уменьшении количества растет процент возврата кредита в срок. Значит, есть зависимость между количеством детей и возвратом кредита в срок.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Аналогично составим следующие сводные таблицы. Семейное положение - столбец family_status

data_family = data.pivot_table(index = 'family_status', values='debt', aggfunc = ('sum', 'count', 'mean')) 


data_family['mean'] = data_family['mean'].apply(per) 
data_family = data_family.rename(columns = {'sum': 'Количество должников', 'count': 'Общее количество', 'mean': 'Среднее по задолжностям'}) 
data_family.sort_values(by='Среднее по задолжностям', ascending=False)

,Общее количество,Среднее по задолжностям,Количество должников
family_status,,,
Не женат / не замужем,2796.0,9.76%,273.0
гражданский брак,4134.0,9.31%,385.0
женат / замужем,12261.0,7.56%,927.0
в разводе,1189.0,7.06%,84.0
вдовец / вдова,951.0,6.62%,63.0


**Вывод:** 

Есть зависимость между семейным положением и возвратом кредита в срок. Люди, которые не вступали в брак хуже всего возвращают в срок. Лучше них возвращают люди в браке. По первым трем строчкам можно увидеть зависимость и рассмотреть это со стороны повышения уровня ответственности. У людей, вступивших в брак, она выше.

Мало данных для людей в разводе и вдов/вдовцов, но при учете этих статусов можно сделать заключение о том, что те, кто был или состоит в браке платят в срок.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# В качестве индекса был выбран заранее подготовленный столбец total_income_category 
# В нем были категоризированы доходы на A, B, C, D, E
# Условия категоризации - п.2.6, задание 16

data_income = data.pivot_table(index = 'total_income_category', values='debt', aggfunc = ('sum', 'count', 'mean')) 


data_income['mean'] = data_income['mean'].apply(per) 
data_income = data_income.rename(columns = {'sum': 'Количество должников', 'count': 'Общее количество', 'mean': 'Среднее по задолжностям'}) 
data_income.sort_values(by='Среднее по задолжностям', ascending=False)

,Общее количество,Среднее по задолжностям,Количество должников
total_income_category,,,
E,22.0,9.09%,2.0
C,15921.0,8.50%,1353.0
A,25.0,8.00%,2.0
B,5014.0,7.06%,354.0
D,349.0,6.02%,21.0


In [ ]:
data_income['mean', 'debt'] = data_income['mean', 'debt'].apply(per)
data_income

**Вывод:** 

Для категорий E, A, D мало данных, зависимость была рассмотрена для категорий B и C. В таком случае заемщики с доходом 50001-200000 платят хуже тех, у кого доход составляет 200001-1000000.

Здесь трудно сказать о зависимости, так как при обращении заемщика банки сопоставляют его доход со средним доходом специалистов того же профиля и регионом. Перед выводом о зависимости лучше всего провести еще одно исследование с учетом этих факторов.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Цели кредита также были разделены на 4 категории
# Условия категоризации - п.2.6, задание 18

data_purpose = data.pivot_table(index = 'purpose_category', values='debt', aggfunc = ('sum', 'count', 'mean')) 


data_purpose['mean'] = data_purpose['mean'].apply(per) 
data_purpose = data_purpose.rename(columns = {'sum': 'Количество должников', 'count': 'Общее количество', 'mean': 'Среднее по задолжностям'}) 
data_purpose.sort_values(by='Среднее по задолжностям', ascending=False)

,Общее количество,Среднее по задолжностям,Количество должников
purpose_category,,,
операции с автомобилем,4279.0,9.35%,400.0
получение образования,3988.0,9.25%,369.0
проведение свадьбы,2313.0,7.91%,183.0
операции с недвижимостью,10751.0,7.26%,780.0


In [ ]:
data_purpose['mean', 'debt'] = data_purpose['mean', 'debt'].apply(per)
data_purpose

**Вывод:** 

Люди, которые берут кредит с целью операций с недвижимостью лучше всего вовзращают в срок. Это может быть связано с тем, что они берут его более осознанно из-за объема покупки. Также хорошо возвращают люди, которые берут кредит на свадьбу.

Хуже всего возвращают кредит люди, которые берут с целью получения образования или покупки автомобиля.

#### 3.5 Возможные причины появления пропусков в исходных данных.

*Ответ:* 

Причинами появления пропусков в исходных данных могут быть:
- ошибка при вводе данных
- намеренно сделанный пропуск в данных
- из-за отсутствия данных в других столбца


## Итоги исследования

Перед исследованием были получены данные от банка — статистика о платёжеспособности клиентов. В данных были пропуски, дубликаты. После их предобработки все было устранено. Данные по доходам клиентов и целям взятия кредитов были категоризированы.

В ходе исследования надежности заемщиков было выялено, что есть зависимость между возвращением кредитов в срок и количеством детей, также между семейным статусом. В таком случае для модели самого надежного заемщика можно выделить следующие аспекты:
- нет детей
- в браке на данный момент/состоял в браке
- кредит с целью покупки недвижимости

Можно добавить предпочительный доход, но для более точного результата следует выявить предпочтительный доход с учетом дохода и профиля специализации заемщика.